## Explore one-hit vs. two-hit samples in expression space

In [1]:
from pathlib import Path
import pickle as pkl

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import sys; sys.path.append('..')
import config as cfg
from data_utilities import load_cnv_data

%load_ext autoreload
%autoreload 2

In [2]:
# park et al. geneset info
park_loss_data = cfg.data_dir / 'park_loss_df.tsv'
park_gain_data = cfg.data_dir / 'park_gain_df.tsv'

# park et al. significant gene info
park_loss_sig_data = cfg.data_dir / 'park_loss_df_sig_only.tsv'
park_gain_sig_data = cfg.data_dir / 'park_gain_df_sig_only.tsv'

# park et al. gene/cancer type predictions
park_preds_dir = cfg.data_dir / 'park_genes_all_preds'

# mutation and copy number data
pancancer_pickle = Path('/home/jake/research/mpmp/data/pancancer_data.pkl')

### Load mutation info

For now, just use binary mutation status from the pancancer repo. In the future we could pull more granular info from MC3, but it would take some engineering of `1_get_mutation_counts` to do this for lots of genes.

In [3]:
park_loss_df = pd.read_csv(park_loss_data, sep='\t', index_col=0)
park_loss_df.head()

,Gene,Tissue,Effect,Pval,FDR,LogFDR,Target,classification
ACVR1_UCEC,ACVR1,UCEC,0.000000,0.997445,0.681521,0.166514,B_Target,Oncogene
ACVR2A_COADREAD,ACVR2A,COADREAD,0.000000,0.997631,0.681315,0.166646,B_Target,TSG
ACVR2A_LIHC,ACVR2A,LIHC,-1.000178,0.040008,0.122307,0.912515,B_Target,TSG
AJUBA_HNSC,AJUBA,HNSC,-0.936050,0.008733,0.045816,1.338886,A_Hit,TSG
AKT1_BRCA,AKT1,BRCA,0.000000,0.997163,0.681829,0.166318,B_Target,Oncogene


In [4]:
park_gain_df = pd.read_csv(park_gain_data, sep='\t', index_col=0)
park_gain_df.head()

,Gene,Tissue,Effect,Pval,FDR,LogFDR,Target,classification
ACVR1_UCEC,ACVR1,UCEC,-0.482699,0.370095,0.571956,0.242630,B_Target,Oncogene
ACVR2A_COADREAD,ACVR2A,COADREAD,0.187531,0.753981,0.780954,0.107369,B_Target,TSG
ACVR2A_LIHC,ACVR2A,LIHC,0.163421,0.834858,0.821548,0.085362,B_Target,TSG
AJUBA_HNSC,AJUBA,HNSC,0.711518,0.026392,0.147049,0.832509,B_Target,TSG
AKT1_BRCA,AKT1,BRCA,0.535765,0.092176,0.312738,0.504805,B_Target,Oncogene


In [5]:
with open(pancancer_pickle, 'rb') as f:
    pancancer_data = pkl.load(f)

In [6]:
# get (binary) mutation data
# 1 = observed non-silent mutation in this gene for this sample, 0 otherwise
mutation_df = pancancer_data[1]
print(mutation_df.shape)
mutation_df.iloc[:5, :5]

(9074, 20938)


,5S_rRNA,A1BG,A1CF,A2M,A2ML1
SAMPLE_BARCODE,,,,,
TCGA-02-0047-01,0,0,0,0,0
TCGA-02-0055-01,0,0,0,0,0
TCGA-02-2483-01,0,0,0,0,0
TCGA-02-2485-01,0,0,0,0,0
TCGA-02-2486-01,0,0,0,0,0


### Load copy number info

Get copy loss/gain info directly from GISTIC "thresholded" output. This should be the same as (or very similar to) what the Park et al. study uses.

In [7]:
sample_freeze_df = pancancer_data[0]
copy_samples = set(sample_freeze_df.SAMPLE_BARCODE)
print(len(copy_samples))

9074


In [8]:
copy_loss_df, copy_gain_df = load_cnv_data(
    cfg.data_dir / 'pancan_GISTIC_threshold.tsv',
    copy_samples
)
print(copy_loss_df.shape)
copy_loss_df.iloc[:5, :5]

(9068, 25128)


Gene Symbol,ACAP3,ACTRT2,AGRN,ANKRD65,ATAD3A
TCGA-02-0047-01,0,0,0,0,0
TCGA-02-0055-01,0,0,0,0,0
TCGA-02-2483-01,1,1,1,1,1
TCGA-02-2485-01,0,0,0,0,0
TCGA-02-2486-01,0,0,0,0,0


In [9]:
print(copy_gain_df.shape)
copy_gain_df.iloc[:5, :5]

(9068, 25128)


Gene Symbol,ACAP3,ACTRT2,AGRN,ANKRD65,ATAD3A
TCGA-02-0047-01,0,0,0,0,0
TCGA-02-0055-01,1,1,1,1,1
TCGA-02-2483-01,0,0,0,0,0
TCGA-02-2485-01,0,0,0,0,0
TCGA-02-2486-01,0,0,0,0,0


In [10]:
sample_freeze_df.head()

,PATIENT_BARCODE,SAMPLE_BARCODE,DISEASE,SUBTYPE
0,TCGA-OR-A5J1,TCGA-OR-A5J1-01,ACC,Not_Applicable
1,TCGA-OR-A5J2,TCGA-OR-A5J2-01,ACC,Not_Applicable
2,TCGA-OR-A5J3,TCGA-OR-A5J3-01,ACC,Not_Applicable
3,TCGA-OR-A5J5,TCGA-OR-A5J5-01,ACC,Not_Applicable
4,TCGA-OR-A5J6,TCGA-OR-A5J6-01,ACC,Not_Applicable


### Get sample info and groups for gene/cancer type

In [11]:
def get_hits_for_gene_and_tissue(identifier, cancer_classification):
    """Given a gene and tissue, load the relevant mutation/CNV information,
    and divide the samples into groups to compare survival.
    """
    # get patient ids in given cancer type 
    gene, tissue = identifier.split('_')
    tissue_ids = (sample_freeze_df
        .query('DISEASE == @tissue')
        .SAMPLE_BARCODE
    )
    
    # get mutation and copy status
    mutation_status = mutation_df.loc[tissue_ids, gene]
    if cancer_classification == 'TSG':
        copy_status = copy_loss_df.loc[tissue_ids, gene]
    elif cancer_classification == 'Oncogene':
        copy_status = copy_gain_df.loc[tissue_ids, gene]
        
    # get hit groups from mutation/CNV data
    two_hit_samples = (mutation_status & copy_status).astype(int)
    one_hit_samples = (mutation_status ^ copy_status).astype(int)
        
    return pd.DataFrame(
        {'one_hit': one_hit_samples,
         'two_hit': two_hit_samples}
    )

In [12]:
sample_mut_df = get_hits_for_gene_and_tissue('ATRX_LGG', 'TSG')

print(sample_mut_df.shape)
sample_mut_df.iloc[:5, :5]

(507, 2)


,one_hit,two_hit
SAMPLE_BARCODE,,
TCGA-CS-6669-01,0,0
TCGA-CS-4938-01,1,0
TCGA-CS-4941-01,0,0
TCGA-CS-4942-01,1,0
TCGA-CS-4943-01,1,0


In [13]:
sample_mut_df.sum()

one_hit    231
two_hit     22
dtype: int64